In [ ]:



from lume_services.flows.flow_of_flows import FlowOfFlows
from lume_services.services.files.filesystems.mounted import MountedFilesystem
from lume_services.services.scheduling.backends.docker import DockerRunConfig
from lume_services.tests.files import FLOW_OF_FLOWS_YAML
from lume_services.tests.files.flows.flow1 import flow as flow1
from lume_services.tests.files.flows.flow2 import flow as flow2
from lume_services.tests.files.flows.flow3 import flow as flow3
from lume_services import config

import os
import logging

from lume_services.tests.scheduling.test_flows import project_name

logging.basicConfig(level="DEBUG")
#logger = logging.getLogger(__name__)
#logger.setLevel(logging.DEBUG)
#lume_services_logger = logging.getLogger("lume_services.services.scheduling.backends.server")
#lume_services_logger.setLevel(logging.DEBUG)

os.environ["LUME_BACKEND"] = "docker"
os.environ["LUME_MOUNTED_FILESYSTEM__IDENTIFIER"] = "mounted"
os.environ["LUME_MOUNTED_FILESYSTEM__MOUNT_PATH"] = "/Users/jgarra/sandbox/lume-services"
os.environ["LUME_MOUNTED_FILESYSTEM__MOUNT_ALIAS"] = "/User/my_user/data"
mounted_filesystem =  MountedFilesystem(
        mount_path="/Users/jgarra/sandbox/lume-services", mount_alias="/User/my_user/data", identifier="mounted"
    )

lume_services_settings =  config.LUMEServicesSettings()

config.configure(lume_services_settings)


from lume_services.config import context

scheduling_service = context.scheduling_service()
backend = scheduling_service.backend

backend.create_project("test")

flow1_id = backend.register_flow(
        flow1,
        project_name="test",
        image="build-test:latest",
        labels=["lume-services"],
    )


flow2_id = backend.register_flow(
        flow2,
        project_name="test",
        image="build-test:latest",
        labels=["lume-services"],
    )

flow3_id = backend.register_flow(
        flow3,
        project_name="test",
        image="build-test:latest",
        labels=["lume-services"],
    )


filename = f"{mounted_filesystem.mount_alias}/test_docker_backend.txt"
data1 = {
    "text1": "hey",
    "text2": " you",
    "filename": filename,
    "filesystem_identifier": "mounted",
}


data2 = {"file_rep": {"filename": filename, "filesystem_identifier": "mounted","file_type_str": "lume_services.files.file:TextFile"}}
data3 = {"text1": "hey you", "text2": "hey you"}


flow_of_flows = FlowOfFlows.from_yaml(
    FLOW_OF_FLOWS_YAML, scheduling_service=scheduling_service
)
flow_of_flows.compose()
flow_id = flow_of_flows.register()



lume_env = {name: val for name, val in os.environ.items() if "LUME" in name}
# Need to convert to docker network hostnames
lume_env["LUME_RESULTS_DB__HOST"] = "mongodb"
lume_env["LUME_MODEL_DB__HOST"] = "mysql"

host_config = {
    "mounts":  {
                "target": mounted_filesystem.mount_alias,
                "source": mounted_filesystem.mount_path,
                "type": "bind",
            }
}

data = {'flow1-filesystem_identifier': "mounted", 'flow1-filename': filename, 'flow1-text1': "hey", 'flow1-text2': " you", 'flow3-text2': "hey you"}

run_config = DockerRunConfig(
        image="build-test:latest", env=lume_env, host_config=host_config
    )

backend.run_and_return(flow_id = flow_id, parameters=data, run_config = run_config)


